## annotate query

* 各queryに対して、どの企業に関する質問かをアノテーションする

### 設定

In [1]:
!pip install python-dotenv
!pip install pandas
!pip install langchain-core==0.3.0
!pip install langchain-openai==0.2.0


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
load_dotenv(verbose=True)
dotenv_path = ".env"
load_dotenv(dotenv_path)

import pandas as pd
pd.set_option('display.max_rows', 500)

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

### データロード

In [3]:
df_query = pd.read_csv("../data/query/query.csv")
list_query = df_query["problem"].to_list()

### 定数

In [4]:
LIST_COMPANY_NAME = [
    "カゴメ株式会社",
    "グローリー株式会社",
    "サントリーホールディングス株式会社",
    "ハウス食品グループ本社株式会社",
    "パナソニックホールディングス株式会社",
    "全国保証株式会社",
    "日清食品ホールディングス株式会社",
    "日産自動車株式会社",
    "明治ホールディングス株式会社",
    "東急不動産ホールディングス株式会社",
    "東洋エンジニアリング株式会社",
    "株式会社IHI",
    "株式会社キッツ",
    "株式会社クレハ",
    "株式会社メディアドゥ",
    "株式会社モスフードサービス",
    "株式会社4℃ホールディングス",
    "株式会社ライフコーポレーション",
    "株式会社高松コンストラクショングループ",
]

In [5]:
PROMPT_LIST_COMPANY_NAME = "\n".join(LIST_COMPANY_NAME)

### アノテーション

In [6]:
class Annotation(BaseModel):
    company_name: str = Field(description="企業名")
    reason: str = Field(description="企業名を選択した理由")

In [7]:
def annotate(query: str):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "ユーザは以下の企業に関するレポートを保有しています。\n"
                "ユーザが入力した質問について、どの企業レポートを参照すべきか答えてください。\n"
                "株式会社も企業名に含めてください。\n"
                "わからない場合は「不明」と答えてください\n"
                "ユーザが保有している企業レポート:\n"
                "{list_company_name}",
            ),
            (
                "human",
                "{query}",
            )
        ]
    )      

    model = ChatOpenAI(model="gpt-4o-mini")

    chain = prompt | model.with_structured_output(Annotation)

    annotation = chain.invoke({
        "list_company_name": PROMPT_LIST_COMPANY_NAME,
        "query": query,
    })

    return annotation

In [8]:
list_annotation = []
for query in list_query:
    annotation = annotate(query)
    list_annotation.append(annotation)
df_annotation = pd.DataFrame([a.dict() for a in list_annotation])

C:\Users\MikAn\AppData\Local\Temp\ipykernel_8712\448762748.py:5: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  df_annotation = pd.DataFrame([a.dict() for a in list_annotation])


In [9]:
df_annotation.groupby("company_name").size()

company_name
カゴメ株式会社                5
グローリー株式会社              5
サントリーホールディングス株式会社      6
ハウス食品グループ本社株式会社        5
パナソニックホールディングス株式会社     5
不明                     1
全国保証株式会社               5
日清食品ホールディングス株式会社       7
日産自動車株式会社              5
明治ホールディングス株式会社         5
東急不動産ホールディングス株式会社      5
東洋エンジニアリング株式会社         5
株式会社4℃ホールディングス         5
株式会社IHI                5
株式会社キッツ                5
株式会社クレハ                6
株式会社メディアドゥ             5
株式会社モスフードサービス          5
株式会社ライフコーポレーション        5
株式会社高松コンストラクショングループ    5
dtype: int64

In [10]:
df_annotation[~df_annotation["company_name"].isin(LIST_COMPANY_NAME)]

,company_name,reason
95,不明,セグメント別営業利益に関する具体的なデータが含まれている企業レポートは特定できないため。


In [11]:
df_query_annotated = pd.concat([df_query, df_annotation], axis=1)
df_query_annotated

,index,problem,company_name,reason
0,0,高松コンストラクショングループの2025年3月期の受注高の計画は前期比何倍か、小数第三位を四...,株式会社高松コンストラクショングループ,高松コンストラクショングループの受注高の計画に関する情報を参照する必要があるため。
1,1,株式会社キッツの取締役の報酬のうち株式報酬の割合は何％？,株式会社キッツ,ユーザが保有している企業レポートに株式会社キッツが含まれているため
2,2,グローリーが2024年の統合報告書の中で、研究開発費が増加した理由として挙げている事由は何？,グローリー株式会社,ユーザが保有しているレポートの中にグローリー株式会社の企業レポートが含まれているため
3,3,高松コンストラクショングループの2024年3月期の非常勤監査役は何人？,株式会社高松コンストラクショングループ,ユーザが保有している企業レポートに該当する情報が含まれているため
4,4,2023年で即席めんの一人当たりの年間消費量が最も多い国はどこか。,日清食品ホールディングス株式会社,日清食品は即席めんの主要なメーカーであり、消費量に関する情報を持っている可能性が高いから。
5,5,株式会社キッツで2023年度のデジタル教育受講者数は何人？,株式会社キッツ,ユーザが保有している企業レポートの中に株式会社キッツに関する情報が含まれているため。
6,6,2023年度の日清食品ホールディングスの海外事業において、コア営業利益が2番目に高い地域に含...,日清食品ホールディングス株式会社,2023年度の日清食品ホールディングスの海外事業に関する情報を提供するため。
7,7,株式会社キッツの2023年度売上高構成比が2番目に高い事業の製造・販売会社を全て上げよ。（回...,株式会社キッツ,2023年度売上高構成比に関する情報を得るために株式会社キッツのレポートを参照する必要があるため。
8,8,IHIの2023年度の資源・エネルギー・環境事業領域の営業利益が過去最高を記録した要因として...,株式会社IHI,ユーザが保有している企業レポートの中に株式会社IHIのレポートがあるため。
9,9,IHIグループの経営目標の進捗の2022年度と2023年度を比較したとき、営業利益率と税引後...,株式会社IHI,IHIグループの経営目標に関する情報は株式会社IHIのレポートを参照する必要があります。


In [12]:
df_query_annotated.to_csv("../data/query/query_annotated.csv", index=False)